# MaFaulDa
https://www02.smt.ufrj.br/~offshore/mfs/page_01.html

In [ ]:
from zipfile import ZipFile
from pprint import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import find_peaks

import sys
sys.path.append('../../')
from feature import mafaulda as src

MAFAULDA_PATH = '../../../datasets/MAFAULDA.zip'

View some file names in machinery database and display their count.

In [ ]:
files = src.get_mafaulda_files(ZipFile(MAFAULDA_PATH))
print(f'Measurements: {len(files)}')
pprint(files[:10])

Show details of one measurments

In [ ]:
misalign = src.csv_import(ZipFile(MAFAULDA_PATH), 'horizontal-misalignment/1.5mm/16.7936.csv')
misalign.describe()
misalign

Plot tachometer pulses and calculate revolutions per minute (rpm)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))
g = misalign['tachometer'][:0.5].plot(ax=ax[0], title='Tachometer', xlabel='Time [s]', ylabel='Impulse')
g = misalign['rpm'].plot(ax=ax[1], title='RPM', xlabel='Time [s]', ylabel='RPM')

Plot histograms for each column to get sense of the distributions

In [ ]:
g = misalign.hist(figsize=(15, 10), bins=50)

Plot short segment of time domain signal

In [ ]:
x = 1024
l = 4096
g = misalign[['ax', 'ay', 'az']].iloc[x:x+l].plot(subplots=True, figsize=(15, 5), grid=True)

Plot subsampled time domain signal with subsampling factor

In [ ]:
factor = 100
g = misalign[['ax', 'ay', 'az']].iloc[::factor, :].plot(subplots=True, figsize=(15, 5), grid=True)

Plot magnitutes of each acceleration vector

In [ ]:
g = misalign[['mag_a', 'mag_b']].plot(subplots=True, figsize=(10, 5), grid=True)

In [ ]:
src.normality_tests(misalign)

In [ ]:
src.axis_spectrograms(misalign)

Do not use magnitude - it is not oscilatory. Various axis have different responses, but the signal in each axis is **stationary**

In [ ]:
subsample = 50
src.rms_orbitals(misalign.iloc[::subsample, :])

Load normal conditions

In [ ]:
zip_file = ZipFile(MAFAULDA_PATH)
filenames = src.get_mafaulda_files(zip_file)
normal_files = [
    name for name in filenames if name.startswith('normal')
]
normal_cond = src.import_files(zip_file, normal_files, src.csv_import)
normal_cond.head()

Calculate time domain features on normal conditions in one accelerometer axis

In [ ]:
td_featues = src.time_domain_features(normal_cond, 'ax')
td_featues.head(10)

In [ ]:
#sns.pairplot(td_featues)

In [ ]:
sns.heatmap(td_featues.corr(), annot=True)

In [ ]:
imbalance = src.csv_import_td_features(zip_file, 'imbalance/10g/56.9344.csv')
#imbalance.to_csv('imbalance_features.csv')
imbalance.reset_index(inplace=True)

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(projection='3d')
ax.scatter(imbalance['mean'], imbalance['std'], imbalance['rms'])

ax.set_xlabel('Mean')
ax.set_ylabel('Standard deviation')
ax.set_zlabel('Amplitude')

plt.show()

In [ ]:
from multiprocessing.pool import ThreadPool

pool = ThreadPool(processes=4)
imbalance_files = [
    name for name in filenames if name.startswith('imbalance')
]
WINDOW = 2**14
src.resolution_calc(src.FS_HZ, WINDOW)
spectra = src.fft_csv_import(zip_file, 'imbalance/10g/56.9344.csv', fs=src.FS_HZ, window=WINDOW, overlap=0.5, is_welch=True)
spectra.head(10)

In [ ]:
spectra.iloc[0].loc[:500].plot(legend=False)

In [ ]:
np.log(spectra.iloc[0]).iloc[:200].plot(legend=False, grid=True)

In [ ]:
WINDOW = 2**13
from tqdm.notebook import tqdm

zip_file = ZipFile(MAFAULDA_PATH)
imbalancePSD = pd.concat([
    pool.apply_async(src.fft_csv_import, (zip_file, name, WINDOW, 0.5, src.FS_HZ, True)).get()
    for name in tqdm(imbalance_files)
])

In [ ]:
imbalancePSD

In [ ]:
imbalancePSD.T[(10, '56.9344.csv')].loc[:500].plot()

In [ ]:
bins = imbalancePSD.T[(10, '56.9344.csv')]
peaks, properties = find_peaks(bins, prominence=0.01)
plt.plot(bins.index, bins)
plt.scatter(bins.index[peaks], bins[bins.index[peaks]], color='r')
plt.xlim(0, 1000)
plt.xlabel('Frequency [Hz]')
plt.ylabel('Amplitude')

In [ ]:
peak_features = src.extract_peaks(imbalancePSD)
sns.lmplot(x='y_f0', y='y_f1', data=peak_features, fit_reg=False, hue='load_f0', legend=False)
plt.legend(loc='lower right')
plt.show()

In [ ]:
sns.lmplot(x='f_f0', y='f_f1', data=peak_features, fit_reg=False, hue='load_f0', legend=False)
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Spectral statistics
from scipy.stats import skew, kurtosis

rms = lambda x: np.sqrt((x ** 2).mean())
spectral_centroid = lambda x: np.average(x.index, weights=x)

fd_features = pd.concat([
    imbalancePSD.mean(axis=1).rename('mean'),
    imbalancePSD.std(axis=1).rename('std'),
    imbalancePSD.T.apply(lambda x: skew(x)).rename('skew'),
    imbalancePSD.T.apply(lambda x: kurtosis(x)).rename('kurtosis'),
    imbalancePSD.T.apply(rms).rename('rms'),
    imbalancePSD.T.apply(spectral_centroid).rename('centroid')
    ],
    axis=1
)
fd_features.head(10)

In [ ]:
# sns.pairplot(fd_features)

In [ ]:
fd_plain = fd_features.reset_index()
sns.lmplot(x='rms', y='kurtosis', data=fd_plain, fit_reg=False, hue='load', legend=False)
plt.legend(loc='lower right')
plt.show()

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(projection='3d')
ax.scatter(fd_plain['kurtosis'], fd_plain['centroid'], fd_plain['rms'])

ax.set_xlabel('Kurtosis')
ax.set_ylabel('Centroid')
ax.set_zlabel('RMS')

In [ ]:
# Compare different faults

### PSD of severe faults by axis (Welch) - up to 2 kHz (120 000 RPM)

In [ ]:
FS = 50000
WINDOW = 2**13
fault_files = [
    'horizontal-misalignment/2.0mm/60.8256.csv',
    'vertical-misalignment/1.90mm/61.44.csv',
    'imbalance/35g/56.7296.csv',
    'normal/61.44.csv',
    'overhang/ball_fault/35g/32.1536.csv',
    'overhang/cage_fault/35g/54.0672.csv',
    'overhang/outer_race/35g/53.4528.csv',
    'underhang/ball_fault/35g/50.7904.csv',
    'underhang/cage_fault/35g/56.5248.csv',
    'underhang/outer_race/35g/58.9824.csv'
]

src.fft_csv_import_by_axis(ZipFile(MAFAULDA_PATH), 'normal/61.44.csv', 'az', WINDOW, 0.5, src.FS_HZ, True)

### Measurement place A - import worst faults and compare each axis's PSD

In [ ]:
zip_file = ZipFile(MAFAULDA_PATH)
faultPSD_X = pd.concat([
    pool.apply_async(src.fft_csv_import_by_axis, (zip_file, name, 'ax', WINDOW, 0.5, src.FS_HZ, True)).get()
    for name in tqdm(fault_files)
])

faultPSD_Y = pd.concat([
    pool.apply_async(src.fft_csv_import_by_axis, (zip_file, name, 'ay', WINDOW, 0.5, src.FS_HZ, True)).get()
    for name in tqdm(fault_files)
])

faultPSD_Z = pd.concat([
    pool.apply_async(src.fft_csv_import_by_axis, (zip_file, name, 'az', WINDOW, 0.5, src.FS_HZ, True)).get()
    for name in tqdm(fault_files)
])

In [ ]:
f_cutoff = 2000

faultPSD_X_v = faultPSD_X.drop('rpm', axis=1)
x_psd = (
    faultPSD_X_v
    .T[faultPSD_X_v.T.index < f_cutoff]
)
axis = x_psd.plot(
    subplots=True,
    figsize=(20, 15),
    xlabel='Frequency [Hz]',
    ylabel='Amplitude'
)

# Graph RPM
for ax, rpm  in zip(axis, faultPSD_X['rpm']):
    f0 = rpm / 60
    ax.axvline(x=f0, color='red')
    # n - harmonics of rotating frequency
    n = 6
    for i in range(2, n):
        ax.axvline(x=f0 * i, color='orange')

In [ ]:
faultPSD_Y_v = faultPSD_Y.drop('rpm', axis=1)
y_psd = faultPSD_Y_v.T[faultPSD_Y_v.T.index < f_cutoff]
p = y_psd.plot(subplots=True, figsize=(20, 15))

In [ ]:
faultPSD_Z_v = faultPSD_Z.drop('rpm', axis=1)
z_psd = faultPSD_Z_v.T[faultPSD_Z_v.T.index < f_cutoff]
p = z_psd.plot(subplots=True, figsize=(20, 15))